In [1]:
"""
@Title: 金融中的线性问题
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-10-18 21:53:24
@Description: 
"""

'\n@Title: 金融中的线性问题\n@Author(s): Stephen CUI\n@LastEditor(s): Stephen CUI\n@CreatedTime: 2023-10-18 21:53:24\n@Description: \n'

## 因子模型的多元线性回归

In [4]:
import numpy as np
import statsmodels.api as sm

In [7]:
num_periods = 9
all_values = np.array([np.random.random(8) for i in range(num_periods)])

In [12]:
y_values = all_values[:, 0]
x_values = all_values[:, 1:]
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit()

In [17]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     3.187
Date:                Thu, 19 Oct 2023   Prob (F-statistic):              0.407
Time:                        20:54:06   Log-Likelihood:                 14.858
No. Observations:                   9   AIC:                            -13.72
Df Residuals:                       1   BIC:                            -12.14
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0859      0.362      3.002      0.205      -3.510       5.682
x1            -0.5677      0.292     -1.941      0.303      -4.283       3.148
x2            -0.5355      0.288     -1.859      0.314      -4.196       3.125
x3            -0.0137      0.221     -0.062      0.960      -2.818       2.791
x4             0.6347      0.478      1.328      0.411      -5.440       6.709
x5            -0.1030      0.273     -0.378      0.770      -3.566       3.360
x6            -0.6262      0.234     -2.676      0.228      -3.600       2.348
x7             0.0549      0.231      0.238      0.851      -2.876       2.986
==============================================================================
Omnibus:                        1.770   Durbin-Watson:                   2.779
Prob(Omnibus):                  0.413   Jarque-Bera (JB):                0.599
Skew:                          -0.628   Prob(JB):                        0.741
Kurtosis:                       2.859   Cond. No.                         21.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
results.params

array([ 1.0858684 , -0.56770645, -0.53551328, -0.01371734,  0.63469805,
       -0.10295032, -0.62624187,  0.05489536])

## 线性最优化

In [24]:
import pulp

# 申明要求解的变量
x = pulp.LpVariable('x', lowBound=0)
y = pulp.LpVariable('y', lowBound=0)
# 通过文本描述和最优化的的类型来初始化问题
problem = pulp.LpProblem(
    'A_simple_maximization_objective',
    pulp.LpMaximize
)
# 允许使用 += 加入任意数量的限制条件及其描述文本
problem += 3*x + 2*y, 'The objective function'
problem += 2*x + y <= 100, 'lst constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
# 执行线性求解
problem.solve()

1

In [25]:
print('Maximization Results:')
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)

Maximization Results:
x = 20.0
y = 60.0


### 整数规划

In [48]:
dealers = ['X', 'Y', 'Z']
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}

quantities = pulp.LpVariable.dicts('quantity',
                                   dealers,
                                   lowBound=0,
                                   cat=pulp.LpInteger)
# 二进制值，表示是否为某位交易商交易
is_orders = pulp.LpVariable.dicts('orders',
                                  dealers,
                                  cat=pulp.LpBinary)

\begin{equation}
\min \sum_{i=x}^{z}\text{isOrder}_i(\text{variable cost}_i\times \text{quantity}_i+\text{fixed cost}_i)
\end{equation}

In [35]:
model = pulp.LpProblem('A_cost_minimization_problem',
                       pulp.LpMinimize)
model += sum([(variable_costs[i] * quantities[i] + fixed_costs[i])*is_orders[i] for i in dealers]), 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150, 'Total contracts required'
model += 30 <= quantities['X'] <= 100, 'Boundary of total volume of X'
model += 30 <= quantities['Y'] <= 90, 'Boundary of total volume of Y'
model += 30 <= quantities['Z'] <= 70, 'Boundary of total volume of Z'

TypeError: Non-constant expressions cannot be multiplied

In [49]:
model = pulp.LpProblem('A_cost_minimization_problem',
                       pulp.LpMinimize)
model += sum([variable_costs[i] * quantities[i] + fixed_costs[i] * is_orders[i] for i in dealers]), 'Minimize portfolio cost'
model += is_orders['X'] * 30 <= quantities['X'] <= is_orders['X'] * 100, 'Boundary of total volume of X'
model += is_orders['Y'] * 30 <= quantities['Y'] <= is_orders['Y'] * 90, 'Boundary of total volume of Y'
model += is_orders['Z'] * 30 <= quantities['Z'] <= is_orders['Z'] * 70, 'Boundary of total volume of Z'
model += sum([quantities[i] for i in dealers]) == 150, 'Total contracts required'
model.solve()

1

In [51]:
print("Minimization Results:")
for variable in model.variables():
    print(variable, '=', variable.varValue)
print('Total cost:', pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


## 使用矩阵解线性方程组

In [54]:
import numpy as np

A = np.array([[2, 1, 1], 
              [1, 3, 2],
              [1, 0, 0]])
B = np.array([[4], [5], [6]])

In [55]:
np.linalg.solve(A, B)

array([[  6.],
       [ 15.],
       [-23.]])

## LU 分解

In [58]:
import scipy.linalg as linalg
A = np.array([
    [2, 1, 1], 
    [1, 3, 2],
    [1, 0, 0]
])
B = np.array([[4], [5], [6]])
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)
x

array([[  6.],
       [ 15.],
       [-23.]])

In [59]:
import scipy
P, L, U = scipy.linalg.lu(A)

In [60]:
P

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [61]:
L

array([[ 1. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ],
       [ 0.5, -0.2,  1. ]])

In [62]:
U

array([[ 2. ,  1. ,  1. ],
       [ 0. ,  2.5,  1.5],
       [ 0. ,  0. , -0.2]])